#### This code file is same as of 1 ANN training using iterative feature selection process with base model being on common features.

#### Changes made are in this file considers all five output columns for generating a final output binary value. Other hyperparameters too ar changed

In [28]:
#Import all the necessary libraries
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 0

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as nm
nm.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)
# for later versions: 
# tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
# for later versions:
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

#os.environ['TF_DETERMINISTIC_OPS'] = '1'
#os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

# Check GPU availability
print("GPU Available:", tf.config.list_physical_devices("GPU"))

# Check CPU availability
print("CPU Available:", tf.config.list_physical_devices("CPU"))

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler 
from keras.models import Sequential #importing Sequential Model.It uses tensorflow backend and you need to have tensorflow setup. 
from keras.layers import Dense #Importing default dense layer
from sklearn.utils import shuffle
from keras.regularizers import l1, l2
from keras.optimizers import SGD
from sklearn.metrics import accuracy_score


path_dir='C:/Users/subhi/AMNS_Second_phase_ANN_model/Datasets/'
filename1='Common_Data.xlsx'
filename2='Intersection_Data.xlsx'
#Access all the necessary variables and datasets
%store -r common_data 
%store -r random_forest_data
random_forest_data.shape #161 statistical features and one output column
# Using the common_data dataframe created in 1_ANN_Model_on_common_features
#Usibg the appended_data dataframe created in Random_Forest_Code 
common_data

GPU Available: []
CPU Available: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


NARROWFACESECONDARYCOOLINGWATERPRESSUREACTUALMEAN  \
0                                             4.925862   
1                                             4.915455   
2                                             5.482414   
3                                             5.471203   
4                                             5.081299   
..                                                 ...   
586                                           2.140476   
587                                           2.098624   
588                                           2.097216   
589                                           2.091311   
590                                           2.086576   

     MOLDBROADFACE1INLETOUTLETWATERTEMP.DELTAMEAN  STEELLEVELINMOLD-SETMEAN  \
0                                        4.000000                 80.103621   
1                                        4.000000                 80.104899   
2                                        4.000000                 80.106667   
3                                        4.000000                 80.108861   
4                                        4.000000                 80.110282   
..                                            ...                       ...   
586                                      7.994048                 80.108333   
587                                      8.052910                 80.107513   
588                                      7.407216                 80.109175   
589                                      7.121359                 80.106019   
590                                      7.065217                 80.105435   

     SEG0BSECONDARYCOOLINGWATERPRESSUREACTUALMEAN  \
0                                        1.177759   
1                                        1.170657   
2                                        1.239828   
3                                        1.240000   
4                                        1.190000   
..                                            ...   
586                                      0.789524   
587                                      0.334974   
588                                      0.544330   
589                                      0.722087   
590                                      0.859620   

     SEG9+10+11+12BOTTOMSECONDARYCOOLINGWATERFLOWACTUALMEAN  LIQUIDUSTEMPMEAN  \
0                                           190.958678             1521.40790   
1                                           195.140556             1521.76000   
2                                           191.464138             1521.26000   
3                                           187.756139             1521.74000   
4                                           188.373955             1521.41000   
..                                                 ...                    ...   
586                                         213.972321             1514.56430   
587                                         204.833228             1515.94260   
588                                         213.607887             1512.56210   
589                                          31.653689             1512.88320   
590                                          31.550598             1512.30945   

     NARROWFACE1WATERFLOWMEAN  TUNDISHWEIGHTMEAN  OUTPUT  
0                  461.735632          23.854885       1  
1                  461.954545          23.548081       1  
2                  461.534483          24.320115       0  
3                  461.892405          24.275696       0  
4                  462.028249          24.563785       1  
..                        ...                ...     ...  
586                452.720238          23.541369       0  
587                443.820106          27.779471       1  
588                452.283505          22.814948       1  
589                452.150485          24.742087       1  
590                452.614130          24.789022       1  

[591 rows x 9 columns]

### Training the base model 

In [29]:
common_data = shuffle(common_data, random_state=42)
#Defining train and test sizes
total_length= common_data.shape[0]
train_size=int(common_data.shape[0]*0.8)
test_size=common_data.shape[0]-train_size

# First converting dataframe series into numpy array and setting the datatype as float32 for all data
# In second step converting all 1D array into 2D array
train_X=common_data.iloc[:,:-1][0:train_size].to_numpy().astype(nm.float32) #2D array

train_Y=common_data['OUTPUT'][0:train_size].to_numpy().astype(nm.float32) #1D array
train_Y=nm.reshape(train_Y,(train_Y.shape[0],1)) # 2D array

test_X=common_data.iloc[:,:-1][train_size:].to_numpy().astype(nm.float32) #2D array

test_Y=common_data['OUTPUT'][train_size:].to_numpy().astype(nm.float32) #1D array
test_Y=nm.reshape(test_Y,(test_Y.shape[0],1)) # 2D array

print('Dimension of training data',train_X.shape)
print('Dimension of testing data',test_X.shape)

#print(train_X)
print(type(train_X))
print(train_Y.shape)
test_Y.shape #2-D array
#train_Y

Dimension of training data (472, 8)
Dimension of testing data (119, 8)
<class 'numpy.ndarray'>
(472, 1)


(119, 1)

In [30]:
#scaler=MinMaxScaler(feature_range=(0,1)) # scaler object
scaler=RobustScaler()
# First fit the scaler with your training data.
# It will find those parameters using which it is going to scale.
# Then transform any data you want.

train_X=scaler.fit_transform(train_X) 

# Since it has scaled  the data as per the parameters it learnt during training data.
# So it will require 2D array/similiar dimension as of training data.
# So testing data should be a 2D array.
# Testing data only need to be transformed on the basis of the parameters learnt using fit function on training data.

test_X=scaler.transform(test_X) #Scaling the training data and using the same parameters for testing data
#test
#train_X[:]

In [31]:

#Creating a Model
print('Total No of features',len(train_X[0,:])) #No of features
base_model=Sequential() 

base_model.add(Dense(50,input_dim=len(train_X[0,:]),activation='relu',kernel_regularizer=l1(0.01))) # one hidden layer
base_model.add(Dense(1,activation='sigmoid',kernel_regularizer=l1(0.01))) #output layer
        
base_model.summary()

Total No of features 8
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 50)                450       
                                                                 
 dense_9 (Dense)             (None, 1)                 51        
                                                                 
Total params: 501
Trainable params: 501
Non-trainable params: 0
_________________________________________________________________


In [32]:
#model.compile(optimizer='_' , loss='_' , metrics=['accuracy'])
base_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [33]:
#Fitting the training data
#base_model.fit(training data input, training data output, batch_size=_,validation_data=(x_test,y_test))
base_model.fit(train_X,train_Y,epochs=150,batch_size=len(train_X),validation_data=(test_X,test_Y))

# Epochs is number of times i want to do forward and backward propogation.
# batch size is used for update the weights in mini batches. The default back size is 32.
# Validation data : accuracy looking like on validation data after every epoch, you can give test data here too. 
#The loss will be reduced due to backpropgation through time.

#Add regularization or increase number of epochs for better accuracy.

Epoch 1/150
95/95 [==============================] - 1s 4ms/step - loss: 12.2333 - accuracy: 0.5297 - val_loss: 2.3785 - val_accuracy: 0.5630
Epoch 2/150
95/95 [==============================] - 0s 3ms/step - loss: 1.3297 - accuracy: 0.6250 - val_loss: 1.1491 - val_accuracy: 0.6303
Epoch 3/150
95/95 [==============================] - 0s 2ms/step - loss: 1.1983 - accuracy: 0.6653 - val_loss: 1.2813 - val_accuracy: 0.5630
Epoch 4/150
95/95 [==============================] - 0s 2ms/step - loss: 1.2659 - accuracy: 0.6737 - val_loss: 0.9863 - val_accuracy: 0.6218
Epoch 5/150
95/95 [==============================] - 0s 2ms/step - loss: 1.1331 - accuracy: 0.7013 - val_loss: 1.4255 - val_accuracy: 0.6218
Epoch 6/150
95/95 [==============================] - 0s 2ms/step - loss: 1.2191 - accuracy: 0.6864 - val_loss: 0.9275 - val_accuracy: 0.5966
Epoch 7/150
95/95 [==============================] - 0s 3ms/step - loss: 0.9354 - accuracy: 0.6864 - val_loss: 0.8716 - val_accuracy: 0.6387
Epoch 8/150


Epoch 59/150
95/95 [==============================] - 0s 3ms/step - loss: 0.8473 - accuracy: 0.6653 - val_loss: 0.7839 - val_accuracy: 0.6050
Epoch 60/150
95/95 [==============================] - 0s 3ms/step - loss: 0.9992 - accuracy: 0.6610 - val_loss: 0.9299 - val_accuracy: 0.5630
Epoch 61/150
95/95 [==============================] - 0s 2ms/step - loss: 0.9065 - accuracy: 0.6610 - val_loss: 0.9546 - val_accuracy: 0.6050
Epoch 62/150
95/95 [==============================] - 0s 2ms/step - loss: 1.0046 - accuracy: 0.6547 - val_loss: 0.7986 - val_accuracy: 0.6134
Epoch 63/150
95/95 [==============================] - 0s 2ms/step - loss: 0.8367 - accuracy: 0.6631 - val_loss: 0.7704 - val_accuracy: 0.6134
Epoch 64/150
95/95 [==============================] - 0s 2ms/step - loss: 0.8403 - accuracy: 0.6441 - val_loss: 0.8011 - val_accuracy: 0.5798
Epoch 65/150
95/95 [==============================] - 0s 2ms/step - loss: 0.7966 - accuracy: 0.6695 - val_loss: 0.7959 - val_accuracy: 0.6050
Epoch 

95/95 [==============================] - 0s 2ms/step - loss: 0.8304 - accuracy: 0.6758 - val_loss: 0.8203 - val_accuracy: 0.6134
Epoch 117/150
95/95 [==============================] - 0s 2ms/step - loss: 0.7432 - accuracy: 0.6843 - val_loss: 0.9743 - val_accuracy: 0.5882
Epoch 118/150
95/95 [==============================] - 0s 2ms/step - loss: 0.8642 - accuracy: 0.6716 - val_loss: 0.7690 - val_accuracy: 0.6134
Epoch 119/150
95/95 [==============================] - 0s 2ms/step - loss: 0.8303 - accuracy: 0.6822 - val_loss: 0.7669 - val_accuracy: 0.5798
Epoch 120/150
95/95 [==============================] - 0s 3ms/step - loss: 0.7522 - accuracy: 0.6864 - val_loss: 0.8556 - val_accuracy: 0.6050
Epoch 121/150
95/95 [==============================] - 0s 2ms/step - loss: 0.7785 - accuracy: 0.6695 - val_loss: 1.0375 - val_accuracy: 0.5798
Epoch 122/150
95/95 [==============================] - 0s 2ms/step - loss: 0.7639 - accuracy: 0.6886 - val_loss: 0.8876 - val_accuracy: 0.5798
Epoch 123/150

In [96]:
# Create a list to store the selected features and their corresponding accuracies
base_features = common_data.columns.tolist()[:-1]
base_features

['NARROWFACESECONDARYCOOLINGWATERPRESSUREACTUALMEAN',
 'MOLDBROADFACE1INLETOUTLETWATERTEMP.DELTAMEAN',
 'STEELLEVELINMOLD-SETMEAN',
 'SEG0BSECONDARYCOOLINGWATERPRESSUREACTUALMEAN',
 'SEG9+10+11+12BOTTOMSECONDARYCOOLINGWATERFLOWACTUALMEAN',
 'LIQUIDUSTEMPMEAN',
 'NARROWFACE1WATERFLOWMEAN',
 'TUNDISHWEIGHTMEAN']

In [ ]:
# Evaluate the model

testPredict_probs=base_model.predict(test_X)
#testPredict_probs

# Apply threshold to obtain predicted classes
threshold = 0.5  # Adjust this threshold as needed
y_pred = (testPredict_probs > threshold).astype(int)
y_pred

base_accuracy = accuracy_score(test_Y,y_pred)
base_accuracy

### Data for new model

In [ ]:
%store -r feature_imp_RF
total_features=feature_imp_RF["Feature"]
total_features_list= total_features.tolist()

remaining_features=[]
for feature in total_features_list:
    if feature not in base_features:
        remaining_features.append(feature) 
        
len(remaining_features)

### Iterating over remaining features

In [ ]:
new_features=[]
count=0
optimal_features=[]
accuracy_store=[base_accuracy]

# Start the timer
start_time = time.time()

for feature in remaining_features:
    # Add one feature to the feature set
    new_features = base_features + [feature]
    
    #Creating a new data
    new_data=random_forest_data.loc[:,new_features]
    new_data = shuffle(new_data, random_state=42)
    
    #Splitting the new_data into train and test
    new_total_length= new_data.shape[0]
    new_train_size=int(new_data.shape[0]*0.8)
    new_test_size=new_data.shape[0]-train_size
    new_train_X=new_data.iloc[:,:][0:train_size].to_numpy().astype(nm.float32) #2D array
    new_train_Y=common_data['OUTPUT'][0:train_size].to_numpy().astype(nm.float32) #1D array
    new_train_Y=nm.reshape(train_Y,(train_Y.shape[0],1)) # 2D array
    new_test_X=new_data.iloc[:,:][train_size:].to_numpy().astype(nm.float32) #2D array
    new_test_Y=common_data['OUTPUT'][train_size:].to_numpy().astype(nm.float32) #1D array
    new_test_Y=nm.reshape(test_Y,(test_Y.shape[0],1)) # 2D array}

    
    # Create a new model with the updated feature set
    new_model=Sequential() 
    new_model.add(Dense(50,input_dim=len(new_features),activation='relu',kernel_regularizer=l2(0.01))) # one hidden layer
    new_model.add(Dense(1,activation='sigmoid',kernel_regularizer=l2(0.01))) #output layer
        
    # Compile the new model
    new_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    print(count)
    print(feature)
    
    
    # Train the new model
    new_model.fit(new_train_X,new_train_Y,epochs=150,batch_size=5,validation_data=(new_test_X,new_test_Y))
    
    # Evaluate the new model
    testPredict_probs_new=new_model.predict(new_test_X)
    threshold = 0.5  # Adjust this threshold as needed
    y_new_pred = (testPredict_probs_new > threshold).astype(int)
    new_accuracy = accuracy_score(new_test_Y,y_new_pred)
    accuracy_store.append(new_accuracy)
    
    # Update the baseline feature set if the accuracy score improved
    if new_accuracy> base_accuracy:
        base_features = new_features
        base_accuracy = new_accuracy
        optimal_features.append(new_features)
    
    # Check for convergence
    #if len(base_features) == random_forest_data.iloc[:,:-1].shape[1]:
     #   break
        
# Stop the timer and calculate the elapsed time
end_time = time.time()
training_time = end_time - start_time

In [ ]:
print("Simulation ended",training_time)
print("Best feature set:", base_features)
print("Best accuracy score:", base_accuracy)

In [ ]:
accuracy_store

In [ ]:
%store accuracy_store
%store base_features
%store base_accuracy
%store optimal_features

In [ ]:
len(base_features)
base_features

In [ ]:
common_data.columns

In [ ]:
new_features=[]
for feature in remaining_features:
    # Add one feature to the feature set
    new_features = base_features + [feature]

In [ ]:
new_features

In [ ]:
#Creating a new data
new_data=random_forest_data.loc[:,new_features]
new_data = shuffle(new_data, random_state=42)
new_data.shape

In [ ]:
 #Splitting the new_data into train and test
new_total_length= new_data.shape[0]
new_train_size=int(new_data.shape[0]*0.8)
new_test_size=new_data.shape[0]-train_size
new_train_X=new_data.iloc[:,:][0:train_size].to_numpy().astype(nm.float32) #2D array
new_train_Y=common_data['OUTPUT'][0:train_size].to_numpy().astype(nm.float32) #1D array
new_train_Y=nm.reshape(train_Y,(train_Y.shape[0],1)) # 2D array
new_test_X=new_data.iloc[:,:][train_size:].to_numpy().astype(nm.float32) #2D array
new_test_Y=common_data['OUTPUT'][train_size:].to_numpy().astype(nm.float32) #1D array
new_test_Y=nm.reshape(test_Y,(test_Y.shape[0],1)) # 2D array

In [ ]:
new_train_X.shape

In [ ]:
# Create a new model with the updated feature set
new_model=Sequential() 
new_model.add(Dense(50,input_dim=len(new_features),activation='relu',kernel_regularizer=l2(0.01))) # one hidden layer
new_model.add(Dense(1,activation='sigmoid',kernel_regularizer=l2(0.01))) #output layer

In [ ]:
# Compile the new model
new_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    
# Train the new model
new_model.fit(new_train_X,new_train_Y,epochs=150,batch_size=5,validation_data=(new_test_X,new_test_Y))

# Evaluate the new model
testPredict_probs_new=new_model.predict(new_test_X)
threshold = 0.5  # Adjust this threshold as needed
y_new_pred = (testPredict_probs_new > threshold).astype(int)
new_accuracy = accuracy_score(new_test_Y,y_new_pred)
    

In [ ]:
random_forest_data.iloc[:,:-1].shape[1]


In [ ]:
import time

# Compile the new model
new_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Start the timer
start_time = time.time()

# Train the new model
new_model.fit(new_train_X, new_train_Y, epochs=150, batch_size=5, validation_data=(new_test_X, new_test_Y))

# Stop the timer and calculate the elapsed time
end_time = time.time()
training_time = end_time - start_time

# Evaluate the new model
testPredict_probs_new = new_model.predict(new_test_X)
threshold = 0.5  # Adjust this threshold as needed
y_new_pred = (testPredict_probs_new > threshold).astype(int)
new_accuracy = accuracy_score(new_test_Y, y_new_pred)

# Print the training time
print("Training Time: {:.2f} seconds".format(training_time))


In [ ]:
a=[1]
a.append(2)

In [ ]:
a

In [ ]:
a=0.69
b=0.5

In [ ]:
print(a>b)

In [ ]:
print(b>a)